In [2]:
from tasks import *
import cv2

In [3]:
#file name to numeric
def name_to_int(name):
    name = name[:-4]
    return int(name)
#numeric to file name
def int_to_name(digit):
    digit = str(digit)
    i = 6 - len(digit)
    while i > 0:
        digit = '0' + digit
        i -= 1
    return digit + '.jpg'

In [4]:
#directory of original images
directory1 = 'output1/'


In [5]:
Task1 = task1()
detected_img = Task1.get_img()

# execute this after task1a(this will be used for other tasks)

In [7]:
#execute this after task1a

#image name to image array
img_array = {}
for (i,t) in detected_img:
    img_array[i] = t
#counting the boxes in each image
num_people = {}
for (i,t) in detected_img:
    num_people[i] = len(t[1])  

In [6]:
#task1b
directory2 = 'output1b/'
#draw path
#initialize the tracker
track = tracker()
for i in range(1, 796):
    name = int_to_name(i)
    img = os.path.join(directory1, name)
    img = cv2.imread(img)
    track.update(img_array[name][1])
    for e in list(track.object.values()):
        #draw centroid
        img = cv2.circle(img, tuple(e[-1]), 1, (0, 0, 255), 10)
        #draw path of last 10 frames
        if len(e) < 10 and len(e) > 1:
            for i in range(1, len(e)):
                img = cv2.line(img, tuple(e[i - 1]), tuple(e[i]), (0, 0, 255), 1, 4)
        if len(e) >= 10:
            for i in range(len(e) - 10, len(e)):
                img = cv2.line(img, tuple(e[i - 1]), tuple(e[i]), (0, 0, 255), 1, 4)
    #plt.imsave(fname = directory2 + name, arr = cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    cv2.imwrite(directory2 + name, img)

In [20]:
#task1c
directory3 = 'output1c/'
track = tracker(max_disappear = 5)
for i in range(1, 796):
    name = int_to_name(i)
    img = os.path.join(directory1, name)
    img = cv2.imread(img)
    track.update(img_array[name][1])
    track.boxes = num_people[name]   
    #print(track.count)
    count = track.boxes
    #if there's no disappeared object then use number of bounding box at this frame as count
    #other wise use self.count
    for e in track.object_disappear.values():
        if e:
            count = track.count
    #if use bounding box as count then update self.count to this number
    if count != track.count:
        track.count = count
    cv2.putText(img, 'num of people:' + str(count), (50, 50), cv2.FONT_HERSHEY_SIMPLEX,  
                   1, (0, 0, 255), 1, cv2.LINE_AA)
    #plt.imsave(fname = directory2 + name, arr = cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    cv2.imwrite(directory3 + name, img)

In [9]:
#task2
directory4 = 'output2/'
directory2 = 'output1b/'
track = tracker(max_disappear = 1)
#draw task2 box
# if want to change the box, change the track.draw_box(left-top_x, left-top_y, right-bottom_x, right-bottom_y)
#(200, 300) means the  bounding boxes of left-top and the (300, 400) means the bounding boxes of right-bottom.
track.draw_box(200, 300, 300, 400)
for i in range(1, 796):
    name = int_to_name(i)
    img = os.path.join(directory2, name)
    img = cv2.imread(img)
    track.update(img_array[name][1])
    if track.enter:
        cv2.putText(img, 'entering the box:' + str(track.enter), (50, 50), cv2.FONT_HERSHEY_SIMPLEX,  
                   1, (0, 0, 255), 1, cv2.LINE_AA)
    if track.leaving:
        cv2.putText(img, 'leaving the box:' + str(track.leaving), (50, 100), cv2.FONT_HERSHEY_SIMPLEX,  
                   1, (0, 0, 255), 1, cv2.LINE_AA)
    if track.drawed_box:
        x1,y1,x2,y2 = track.drawed_box
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
    # plt.imsave(fname = directory2 + name, arr = cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    cv2.imwrite(directory4 + name, img)

In [10]:
#task3
directory5 = 'output3/'
#draw group
track = tracker()
track_count = tracker(max_disappear = 5)
for i in range(1, 796):
    name = int_to_name(i)
    img = os.path.join(directory1, name)
    img = cv2.imread(img)
    track.update(img_array[name][1])
    track_count.update(img_array[name][1])
    track.update_group(img_array[name][1])
    track_count.boxes = num_people[name]
    count = track_count.boxes
    #if there's no disappeared object then use number of bounding box at this frame as count
    #other wise use self.count
    for e in track_count.object_disappear.values():
        if e:
            count = track_count.count
    #if use bounding box as count then update self.count to this number
    if count != track_count.count:
        track_count.count = count
    num_in_group = 0
    in_group = set()
    for e in list(track.group.values()):
        img = cv2.circle(img, tuple(e['centroid'][-1]), 1, (0, 255, 255), 10)
        x1,y1,x2,y2 = e['box']
        img = cv2.rectangle(img, (x1, y1), (x2, y2), (0,255,255), 2)
        if len(e['objects']) == 1:
            num_in_group += 1
        for o in e['objects']:
            in_group.add(o)
    num_in_group += len(in_group)
    alone = count - num_in_group
    if alone < 0:
        alone = 0
    cv2.putText(img, 'walk alone:' + str(alone), (50, 50), cv2.FONT_HERSHEY_SIMPLEX,  
                   1, (0, 0, 255), 1, cv2.LINE_AA)
    cv2.putText(img, 'walk in group:' + str(num_in_group), (50, 100), cv2.FONT_HERSHEY_SIMPLEX,  
                   1, (0, 0, 255), 1, cv2.LINE_AA)
    # plt.imsave(fname = directory2 + name, arr = cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    cv2.imwrite(directory5 + name, img)

In [11]:
#evaluation Tracking
import os
f = open('gt.txt', 'r')
result = list()
for line in f.readlines():                          
    line = line.strip()                             
    if not len(line):      
        continue    
    lines = list(line.split(","))
    tracker_point = list()
    tracker_point.append(int(lines[0]))
    x = float(lines[2])
    y = float(lines[3])
    w, h = float(lines[4]), float(lines[5])
    centroids = (int(x + w/2), int(y + h/2))
    tracker_point.append(centroids)
    result.append(tracker_point)

#initialize the tracker
track = tracker()
TP = 0
FP = 0
FN = 0
total = 0
correct = 0
for i in range(1, 796):
    name = int_to_name(i)
    img = os.path.join(directory1, name)
    img = cv2.imread(img)
    track.update(img_array[name][1])
    # print(i)
    benchmark = list()
    
    for item in result:
        if i == item[0]:
            benchmark.append(item[1])
    eval_tracking = list()
    for e in list(track.object.values()):
        eval_tracking.append(e[-1])

    # print(eval_tracking)               
    TP, FP, FN, correct, total = eval_task1(benchmark, eval_tracking, TP, FP, FN, correct, total)
evaluation(TP, FP, FN, total, correct)

Precision is 0.894
Recall is 0.901
F1 value is 0.898
Accuracy is 0.898
